# Final Project Exploration

This is our Team's Data Science Tool for Finance final project. In this project we attempt to replicate the results of Table 1 found in Monetary Tightening and US Bank Fragility in 2023: Mark-To-Market Losses and Uninsured Depositor Runs, written by Erica Xuewei Jiang, Gregor Matvos,Tomasz Piskorski, and Amit Seru.

In [260]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import data_read
import load_assets
import calc_functions
import load_WRDS
# import Calc_table_statistic

warnings.filterwarnings("ignore")

### Reports load

- We first import the necessary data from WRDS
- RCFD includes banks with both domestic and foreign branches
- RCON includes banks with only domestic branches
- These data series are divided into series 1 and series 2

In [261]:
rcfd_data_1 = load_WRDS.load_RCFD_series_1()
rcon_data_1 = load_WRDS.load_RCON_series_1()
rcfd_data_2 = load_WRDS.load_RCFD_series_2()
rcon_data_2 = load_WRDS.load_RCON_series_2()

In [262]:
rcfd_data_1.columns = [col.upper() for col in rcfd_data_1.columns]
rcon_data_1.columns = [col.upper() for col in rcon_data_1.columns]
rcfd_data_2.columns = [col.upper() for col in rcfd_data_2.columns]
rcon_data_2.columns = [col.upper() for col in rcon_data_2.columns]

In [264]:
inital_date = '03/31/2022'
analysis_date = '03/31/2023'

## Data Cleaning

### Assets

- We then get assets from both the RCON and RCFD, and combine the dataframes to compute the total/gross asset at bank levels

In [265]:

filtered_asset_level_0 = load_assets.clean_assets(rcfd_data_2,'RCFD2170',inital_date)
filtered_asset_level = load_assets.clean_assets(rcon_data_2,'RCON2170',inital_date)
filtered_asset_level.head()


,Bank_ID,bank_name,report_date,gross_asset
1,37,BANK OF HANCOCK COUNTY,2022-03-31,87842.0
10,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,368875.0
12,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,53236.0
35,354,BISON STATE BANK,2022-03-31,25077.0
41,457,LOWRY STATE BANK,2022-03-31,65336.0


#### Creating a dataframe for total/gross asset for each bank

In [266]:
df_asset = pd.concat([filtered_asset_level_0, filtered_asset_level])
total_asset = df_asset['gross_asset'].sum()
df_asset.head()

,Bank_ID,bank_name,report_date,gross_asset
368,12311,"HUNTINGTON NATIONAL BANK, THE",2022-03-31,176020595.0
855,30810,DISCOVER BANK,2022-03-31,105657408.0
1011,35301,STATE STREET BANK AND TRUST COMPANY,2022-03-31,318494000.0
1745,60143,COMERICA BANK,2022-03-31,89142000.0
1865,63069,CITY NATIONAL BANK,2022-03-31,91770285.0


### LOANS

- We then get assets from both the RCON and RCFD, and combine the dataframes to compute the total/gross asset at bank levels

In [267]:
rcon_cols = ['RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569']
rcon_cols2 = ['RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575']
rcfd_cols = ['RCFDA570', 'RCFDA571', 'RCFDA572', 'RCFDA573', 'RCFDA574', 'RCFDA575']

# Residential Loans with repricing maturity (secured by first liens)
df_loans_first_lien_domestic = load_assets.clean_loans(rcon_data_1,rcon_cols,inital_date)

# Loans and Leases (not-secured by first liens)
df_loans_exc_first_lien = load_assets.clean_loans(rcfd_data_1,rcfd_cols,inital_date)
df_loans_exc_first_lien_domestic = load_assets.clean_loans(rcon_data_2,rcon_cols2,inital_date)

df_loans_first_lien_domestic.head()

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
1,37,BANK OF HANCOCK COUNTY,2022-03-31,355,354,2430,2229,2641,0
13,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,683,588,2566,5361,2137,0
20,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,74,22,2136,4966,30924,113757
28,9553,ADAMS COUNTY BANK,2022-03-31,463,1013,1098,2114,2437,163
30,354,BISON STATE BANK,2022-03-31,41,230,995,4528,441,5310


In [268]:
df_other_loan = pd.concat([df_loans_exc_first_lien_domestic, df_loans_exc_first_lien])
df_other_loan = df_other_loan.sort_index()
df_other_loan.head()

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
1,37,BANK OF HANCOCK COUNTY,2022-03-31,1082.0,3574.0,2928.0,3088.0,2106.0,0.0
10,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,8861.0,11647.0,16963.0,11508.0,18670.0,5189.0
12,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,6436.0,2335.0,1715.0,2749.0,767.0,481.0
35,354,BISON STATE BANK,2022-03-31,1023.0,802.0,1041.0,1249.0,489.0,0.0
41,457,LOWRY STATE BANK,2022-03-31,5623.0,5694.0,6740.0,15109.0,6130.0,1116.0


### RMBs

In [269]:
rmbs_cols = ['RCFDA555', 'RCFDA556', 'RCFDA557', 'RCFDA558', 'RCFDA559', 'RCFDA560']
rmbs_dom_cols = ['RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560']
df_RMBS = load_assets.clean_loans(rcfd_data_1,rmbs_cols,inital_date)
df_RMBS_dom = load_assets.clean_loans(rcon_data_1,rmbs_dom_cols,inital_date) #Form 051 - Domestic

In [270]:
df_RMBS_Final = pd.concat([df_RMBS_dom, df_RMBS])
df_RMBS_Final = df_RMBS_Final.sort_index()
df_RMBS_Final.head()

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
1,37,BANK OF HANCOCK COUNTY,2022-03-31,0.0,0.0,0.0,0.0,856.0,6021.0
13,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0
20,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,2814.0,86.0,0.0,1049.0,26085.0,16737.0
28,9553,ADAMS COUNTY BANK,2022-03-31,43.0,178.0,156.0,760.0,7107.0,10334.0
30,354,BISON STATE BANK,2022-03-31,0.0,0.0,0.0,0.0,58.0,0.0


# Treausurys and Other (non-RMBs)

In [271]:
non_RMBS_cols = ['RCFDA549', 'RCFDA550', 'RCFDA551', 'RCFDA552', 'RCFDA553', 'RCFDA554']
non_RMBS_dom_cols = ['RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554']

df_non_RMBS = load_assets.clean_loans(rcfd_data_2,non_RMBS_cols,inital_date)
df_non_RMBS_dom = load_assets.clean_loans(rcon_data_2,non_RMBS_dom_cols,inital_date)

In [272]:
df_treasury_and_others = pd.concat([df_non_RMBS_dom, df_non_RMBS])
df_treasury_and_others = df_treasury_and_others.sort_index()
df_treasury_and_others.head()

,Bank_ID,bank_name,report_date,<3m,3m-1y,1y-3y,3y-5y,5y-15y,>15y
1,37,BANK OF HANCOCK COUNTY,2022-03-31,0.0,0.0,330.0,708.0,26220.0,19212.0
10,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,0.0,0.0,8190.0,7815.0,16591.0,5854.0
12,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,600.0,784.0,2268.0,2813.0,11784.0,135.0
35,354,BISON STATE BANK,2022-03-31,150.0,348.0,0.0,189.0,0.0,0.0
41,457,LOWRY STATE BANK,2022-03-31,0.0,0.0,952.0,479.0,0.0,0.0


In [273]:
other_cols = ['RCFDA561', 'RCFDA562']
other_cols2 = ['RCONA561', 'RCONA562']
other_names = ['<3y', '>3y']

df_other_MBS =  load_assets.clean_others(rcfd_data_1,other_cols,other_names,inital_date)
df_other_MBS_domestic =  load_assets.clean_others(rcon_data_2,other_cols2,other_names,inital_date)
df_other_MBS_total = pd.concat([df_other_MBS, df_other_MBS_domestic])
df_other_MBS_total.head()

,Bank_ID,bank_name,report_date,<3y,>3y
376,12311,"HUNTINGTON NATIONAL BANK, THE",2022-03-31,905975.0,10885639.0
855,30810,DISCOVER BANK,2022-03-31,0.0,0.0
1013,35301,STATE STREET BANK AND TRUST COMPANY,2022-03-31,10596000.0,23772000.0
1704,112837,"CAPITAL ONE, NATIONAL ASSOCIATION",2022-03-31,2189917.0,17014374.0
2033,171133,EASTERN NATIONAL BANK,2022-03-31,0.0,0.0


In [274]:
other_leases = ['RCFDA247']
other_leases2 = ['RCONA247']
other_names2 = ['<1y']

df_other_loan_lease =  load_assets.clean_others(rcfd_data_1,other_leases,other_names2,inital_date)
df_other_loan_lease_domestic =  load_assets.clean_others(rcon_data_1,other_leases2,other_names2,inital_date)
df_total_other_loan_lease = pd.concat([df_other_loan_lease, df_other_loan_lease_domestic])
df_total_other_loan_lease.head()

,Bank_ID,bank_name,report_date,<1y
376,12311,"HUNTINGTON NATIONAL BANK, THE",2022-03-31,19296859.0
855,30810,DISCOVER BANK,2022-03-31,185091.0
1013,35301,STATE STREET BANK AND TRUST COMPANY,2022-03-31,20053000.0
1704,112837,"CAPITAL ONE, NATIONAL ASSOCIATION",2022-03-31,17539810.0
2033,171133,EASTERN NATIONAL BANK,2022-03-31,50876.0


# Asset Decomposition

- We caculated the total amount of core assets (RMBs, Treasuries, Residential Loans, Other Loans)
- We divided this by total amount of the asset (asset ratio), and calculate the ratios among the 4 categories
- We also calculated the ratio for other MBS and other leases, but these are not used in the subsequent analyses

In [275]:
sum_asset = 0
for df in [df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan]:
    total = df[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
    sum_asset += total

print('total assets:',"{:,.2f}".format(sum_asset))

total assets: 16,590,518,404.00


In [276]:
print('assets ratio:',"{:,.2%}".format(sum_asset/total_asset))

assets ratio: 69.16%


In [277]:
RMBS = df_RMBS_Final[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
a = RMBS / total_asset
"{:,.2%}".format(a)

'9.81%'

In [278]:
df_loans = df_loans_first_lien_domestic[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
b = df_loans / total_asset
"{:,.2%}".format(b)

'9.28%'

In [279]:
df_treasury = df_treasury_and_others[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
c = df_treasury / total_asset
"{:,.2%}".format(c)

'12.26%'

In [280]:
df_other_ln = df_other_loan[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
d = df_other_ln / total_asset
"{:,.2%}".format(d)

'37.81%'

In [281]:
df_other_MBS = df_other_MBS_total[['<3y', '>3y']].sum().sum()
e = df_other_MBS  / total_asset
"{:,.2%}".format(e)

'3.98%'

In [282]:
df_total_other_ll = df_total_other_loan_lease['<1y'].sum()
f = df_total_other_ll / total_asset
"{:,.2%}".format(f)

'9.35%'

In [283]:
"{:,.2%}".format(a+b+c+d+e) #matches with the function above

'73.14%'

## roughly the same as the security component (see Panel A)

In [284]:
"{:,.2%}".format(a+e+c)

'26.05%'

## roughly the same as the total loan component (see Panel A)

In [285]:
"{:,.2%}".format(d+b)

'47.09%'

# Table Analysis (Below are the contents for first column of table 1)

# 1. Aggregate Loss and Bank Loss (include each share)

###  Treasury prices

In [286]:
combined_index_df = data_read.load_df('combined_index_df')
treasury_prices = combined_index_df[['iShares 0-1', 'iShares 1-3', 'sp 3-5', 'iShares 7-10', 'iShares 10-20', 'iShares 20+']]
treasury_prices = treasury_prices.resample('Q').first()
treasury_prices = treasury_prices.loc[inital_date:analysis_date]
treasury_prices

,iShares 0-1,iShares 1-3,sp 3-5,iShares 7-10,iShares 10-20,iShares 20+
date,,,,,,
2022-03-31,102.080002,81.459160,490.30,109.055511,138.736893,139.034607
2022-06-30,101.980003,79.252747,465.35,101.439735,126.306000,124.729774
2022-09-30,101.980003,79.260551,464.47,98.588120,114.192192,109.723854
2022-12-31,102.220001,77.765366,445.74,91.988579,103.304291,97.439491
2023-03-31,103.180000,78.298477,450.77,92.527946,103.718979,95.607788


In [287]:
df_SP_Treasury_bond_index = data_read.load_df('Treasury_index',1)
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.resample('Q').first()
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.loc[inital_date:analysis_date]
df_SP_Treasury_bond_index


,S&P U.S. Treasury Bond Index
date,
2022-03-31,470.35
2022-06-30,449.59
2022-09-30,438.25
2022-12-31,421.94
2023-03-31,423.71


In [288]:
df_iShare_MBS_ETF = data_read.load_df('MBS_ETF',1,csv=True)
df_iShare_MBS_ETF.index = pd.to_datetime(df_iShare_MBS_ETF.index)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.resample('Q').first()
df_iShare_MBS_ETF.index.rename('date', inplace=True)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.loc['2022-03-31':'2023-03-31']
df_iShare_MBS_ETF

,Open,High,Low,Close,Adj Close,Volume
date,,,,,,
2022-03-31,107.279999,107.279999,107.000000,107.000000,100.458702,2352200
2022-06-30,100.940002,101.349998,100.879997,101.260002,95.471237,1854600
2022-09-30,97.989998,98.599998,97.809998,98.010002,92.840919,1463400
2022-12-31,92.169998,92.879997,92.120003,92.220001,87.885315,2229800
2023-03-31,93.559998,93.699997,93.110001,93.190002,89.444168,2060800


In [289]:
RMBS_Multiplier = calc_functions.RMBs_Multiplier(df_SP_Treasury_bond_index, df_iShare_MBS_ETF,inital_date,analysis_date)
RMBS_Multiplier

1.1057098336413254

In [290]:
#Create copies for each dataframe
df_RMBS_Final_1 = df_RMBS_Final.copy() 
df_loans_first_lien_domestic_1 = df_loans_first_lien_domestic.copy()
df_treasury_and_others_1 = df_treasury_and_others.copy()
df_other_loan_1 = df_other_loan.copy()

In [291]:
bank_losses = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_Final, df_loans_first_lien_domestic,\
                                            df_treasury_and_others, df_other_loan, treasury_prices, RMBS_Multiplier, df_asset)
bank_losses

,bank_name,bank_ID,RMBs_loss,treasury_loss,loans_loss,other_loan_loss,total_loss,Share RMBs,Share Treasury and Other,Share Residential Mortgage,Share Other Loan,RMBs_asset,treasury_asset,residential_mortgage_asset,other_loan_asset,core_asset,gross_asset,loss/core_asset,loss/gross_asset
0,"HUNTINGTON NATIONAL BANK, THE",12311,-8.105538e+06,-8.757660e+05,-5.646935e+06,-4.401663e+06,-1.902990e+07,42.593691,4.602052,29.674011,23.130245,23910558.0,6623679.0,22529389,92122592.0,145186218.0,176020595.0,0.131072,0.108112
1,DISCOVER BANK,30810,-9.594459e+04,-1.434050e+05,-2.555414e+05,-1.936466e+06,-2.431357e+06,3.946134,5.898146,10.510240,79.645481,364810.0,5612015.0,804718,92452467.0,99234010.0,105657408.0,0.024501,0.023012
2,STATE STREET BANK AND TRUST COMPANY,35301,-7.345597e+06,-3.128506e+06,0.000000e+00,-3.729909e+05,-1.084709e+07,67.719491,28.841883,-0.000000,3.438626,22436000.0,62518000.0,0,35293000.0,120247000.0,318494000.0,0.090207,0.034057
3,COMERICA BANK,60143,-3.153674e+06,-5.015969e+05,-3.526598e+05,-2.206119e+04,-4.029991e+06,78.255093,12.446600,8.750882,0.547425,9378000.0,4835000.0,1825000,47475000.0,63513000.0,89142000.0,0.063451,0.045209
4,CITY NATIONAL BANK,63069,-7.975196e+04,-4.008263e+05,-4.672095e+06,-1.631691e+06,-6.784364e+06,1.175526,5.908090,68.865624,24.050760,336792.0,5698026.0,19072816,39175374.0,64283008.0,91770285.0,0.105539,0.073928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4839,ONESOUTH BANK,1015832,2.382992e+00,-7.635103e+01,-2.163444e+03,-9.860389e+03,-1.209780e+04,-0.019698,0.631115,17.882951,81.505632,200.0,947.0,21688,106539.0,129374.0,162288.0,0.093510,0.074545
4840,COMMUNITY BANK & TRUST COMPANY,1015841,-2.204866e+03,-1.046772e+04,-2.241963e+03,-8.305230e+03,-2.321978e+04,9.495636,45.081052,9.655403,35.767909,7034.0,64878.0,25904,109531.0,207347.0,281351.0,0.111985,0.082530
4841,LAKE ELMO BANK,1015850,-4.212290e+03,-3.517474e+03,-2.288240e+04,-2.243857e+04,-5.305074e+04,7.940116,6.630395,43.133053,42.296436,18309.0,35251.0,111927,202608.0,368095.0,532386.0,0.144122,0.099647
4842,OZARKS FEDERAL SAVINGS AND LOAN ASSOCIATION,1016174,-4.220012e+03,-1.115032e+02,-3.168615e+04,-8.598832e+03,-4.461650e+04,9.458411,0.249915,71.018911,19.272763,17100.0,2124.0,124787,37838.0,181849.0,240084.0,0.245349,0.185837


In [292]:
median_percentage = bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].median()
median_percentage
#close to reported values, but the distribution looks good

Share RMBs                     7.274657
Share Treasury and Other      17.918497
Share Residential Mortgage    18.028783
Share Other Loan              37.271190
dtype: float64

In [293]:
std_percentages =bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].std()
std_percentages #this is slightly different 

Share RMBs                    16.473288
Share Treasury and Other      22.729545
Share Residential Mortgage    21.397766
Share Other Loan              24.115604
dtype: float64

## Total, median, and std of losses

In [294]:
total_sum_loss = bank_losses['total_loss'].sum()
total_sum_loss

-2030252700.3402686

In [295]:
median_bank_loss = bank_losses['total_loss'].median()
median_bank_loss 

-29677.650127523568

In [296]:
std_bank_loss = bank_losses['total_loss'].std()
std_bank_loss

6515039.202427551

## Checks

In [297]:
core_asset = bank_losses['core_asset'].sum()
core_asset

16590518404.0

In [298]:
gross_asset = bank_losses['gross_asset'].sum()
gross_asset 

23988517644.0

In [299]:
core_asset / gross_asset

0.6916024845807683

# 2. Loss / Asset

## Ratio Results

In [300]:
median_loss_asset_ratio = bank_losses['loss/gross_asset'].median()
median_loss_asset_ratio

0.09394600976254505

In [301]:
average_loss_asset_ratio = bank_losses['loss/gross_asset'].mean()
average_loss_asset_ratio

0.09914503331730583

In [302]:
std_loss_asset_ratio = bank_losses['loss/gross_asset'].std()
std_loss_asset_ratio

0.0490027526563258

# 3. Uninsured Deposit/MM Asset

- Uninsured Deposits are directly reported as RCON5597 in the call reports
- MM Asset: total assets in 2022:Q1 minus the mark-to-market value loss (value) 

In [303]:
uninsured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCON5597']]
uninsured_deposit = uninsured_deposit .rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCON5597': 'uninsured_deposit'

})
uninsured_deposit = uninsured_deposit[uninsured_deposit['report_date'] == '03/31/2022']
uninsured_deposit['uninsured_deposit'].sum()/gross_asset #Similar to Panel A result of 37.4%

0.3579107245147957

In [304]:
uninsured_deposit['uninsured_deposit'].sum() #Roughly 9 trillion 

8585747730.0

In [305]:
def calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses):
    
    # Initialize an empty list to store the results
    results = []
    
    # Adjust the uninsured_deposit DataFrame to use both 'bank_name' and 'Bank_ID' as a multi-index for quick lookup
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Adjust the lookup to include 'Bank_ID'
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate 'MM Asset' as the sum of 'total_loss' and 'gross_asset' (as defined in the paper)
        mm_asset = bank_loss_row['total_loss'] + bank_loss_row['gross_asset']
        
        # Calculate Uninsured Deposit/MM Asset ratio 
        if mm_asset > 0:
            uninsured_deposit_mm_asset_ratio = uninsured_deposit_value / mm_asset
        
        # Append to final dataframe
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id, 
            'total_loss': bank_loss_row['total_loss'], 
            'total_asset': bank_loss_row['gross_asset'],
            'mm_asset': mm_asset,
            'uninsured_deposit': uninsured_deposit_value, 
            'Uninsured_Deposit_MM_Asset': uninsured_deposit_mm_asset_ratio
        })
    
    # Convert results list to DataFrame and sort by 'Bank_ID'
    results_df = pd.DataFrame(results).sort_values(by=['bank_name', 'bank_ID'])
    
    return results_df

un_mm_ratio = calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses)

## Checks

In [306]:
un_mm_ratio['total_loss'].sum()

-2030252700.3402684

In [307]:
un_mm_ratio['total_asset'].sum()

23988517644.0

In [308]:
un_mm_ratio['mm_asset'].sum()

21958264943.65973

## Ratio Results

In [309]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].median()

0.3251550697555542

In [310]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].std()

0.1696668231426178

# 4. Insured Deposit Coverage Ratio


#### we defined Insured Deposit Coverage Ratio as total amount of deposit accounts of:

- RCFDF049 - Deposit accounts (excluding retirement accounts) of 250000 or less
- RCFDF045 - Retirement deposit accounts of 250000 or less

In [311]:
insured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONF049', 'RCONF045']] 
#RCFDF049 are Deposit accounts (excluding retirement accounts) of $250,000 or less
#RCFDF045 are Retirement deposit accounts of $250,000 or less

insured_deposit = insured_deposit.rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONF049': 'insured_deposit_1',
    'RCONF045': 'insured_deposit_2'
})
insured_deposit = insured_deposit[insured_deposit['report_date'] == '03/31/2022']
insured_deposit['insured_deposit_1'].sum()/total_asset + insured_deposit['insured_deposit_2'].sum()/total_asset

#THESE NUMBERS ARE SLIGHLY FAR FROM INSURED DEPOSIT IN PANEL A

0.33296701528357264

In [312]:
insured_deposit['insured_deposit'] = insured_deposit['insured_deposit_1'] + insured_deposit['insured_deposit_2']
insured_deposit 

,bank_ID,bank_name,report_date,insured_deposit_1,insured_deposit_2,insured_deposit
1,37,BANK OF HANCOCK COUNTY,2022-03-31,57430.0,2428.0,59858.0
13,242,FIRST COMMUNITY BANK XENIA-FLORA,2022-03-31,33454.0,1385.0,34839.0
20,279,"MINEOLA COMMUNITY BANK, SSB",2022-03-31,197561.0,11879.0,209440.0
28,9553,ADAMS COUNTY BANK,2022-03-31,87873.0,6269.0,94142.0
30,354,BISON STATE BANK,2022-03-31,9794.0,0.0,9794.0
...,...,...,...,...,...,...
38151,5582846,"SQUARE FINANCIAL SERVICES, INC.",2022-03-31,78420.0,0.0,78420.0
38155,5591633,"PLANTE MORAN TRUST, NATIONAL ASSOCIATION",2022-03-31,NaN,NaN,NaN
38172,38740,OWEN COUNTY STATE BANK,2022-03-31,163878.0,10252.0,174130.0
38174,37640,GERMAN AMERICAN BANK,2022-03-31,2701479.0,69885.0,2771364.0


In [313]:
insured_deposit['insured_deposit'].sum()

7987385121.0

In [314]:
def insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses):
    # Initialize an empty list to store the results
    results = []
    
    # Create dictionaries from insured and uninsured deposits for quick lookup
    insured_lookup = insured_deposit.set_index(['bank_name', 'bank_ID'])['insured_deposit'].to_dict()
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Retrieve insured and uninsured deposit values
        insured_deposit_value = insured_lookup.get((bank_name, bank_id), 0)
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate mark-to-market asset value as the sum of 'total_asset' minus 'total_loss'
        mark_to_market_asset_value = bank_loss_row['gross_asset'] + bank_loss_row['total_loss']
        
        # Calculate the insured deposit coverage ratio
        if insured_deposit_value > 0:  # Prevent division by zero
            coverage_ratio = (mark_to_market_asset_value - uninsured_deposit_value - insured_deposit_value) / insured_deposit_value
        
        # Append the result
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id,
            'mm_asset': mark_to_market_asset_value,
            'insured_deposit': insured_deposit_value,
            'uninsured_deposit': uninsured_deposit_value,
            'insured_deposit_coverage_ratio': coverage_ratio
        })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

coverage_ratios_df = insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses)
coverage_ratios_df

,bank_name,bank_ID,mm_asset,insured_deposit,uninsured_deposit,insured_deposit_coverage_ratio
0,"HUNTINGTON NATIONAL BANK, THE",12311,1.569907e+08,78986097.0,83160026.0,-0.065270
1,DISCOVER BANK,30810,1.032261e+08,57543068.0,8691421.0,0.642850
2,STATE STREET BANK AND TRUST COMPANY,35301,3.076469e+08,11687000.0,155822000.0,11.990922
3,COMERICA BANK,60143,8.511201e+07,21921000.0,51044000.0,0.554127
4,CITY NATIONAL BANK,63069,8.498592e+07,17928917.0,58096403.0,0.499785
...,...,...,...,...,...,...
4839,ONESOUTH BANK,1015832,1.501902e+05,91705.0,NaN,NaN
4840,COMMUNITY BANK & TRUST COMPANY,1015841,2.581312e+05,178818.0,NaN,NaN
4841,LAKE ELMO BANK,1015850,4.793353e+05,303598.0,NaN,NaN
4842,OZARKS FEDERAL SAVINGS AND LOAN ASSOCIATION,1016174,1.954675e+05,165673.0,NaN,NaN


## Checks

In [315]:
coverage_ratios_df['insured_deposit'].sum()

7987385121.0

In [316]:
coverage_ratios_df['uninsured_deposit'].sum()

8585747730.0

In [317]:
coverage_ratios_df['mm_asset'].sum()

21958264943.65973

## Ratio Results

In [318]:
coverage_ratios_df['insured_deposit_coverage_ratio'].median()

0.393694836115703

In [319]:
coverage_ratios_df['insured_deposit_coverage_ratio'].std()

329.3194355896452

# Final Table Results

- For table 1, we used the above methods to aggregate the stastics across losses

In [320]:
def final_statistic_table(bank_losses_assets, uninsured_deposit_mm_asset, insured_deposit_coverage, index_name = 'All Banks'):
    # Merge the DataFrames on bank_name and Bank_ID to include uninsured deposit/MM Asset ratios and insured deposit coverage ratios
    
    
    bank_count = len(bank_losses_assets.index)

    final_stats = pd.DataFrame({
        'Aggregate Loss': [f"{-round(bank_losses_assets['total_loss'].sum() / 1e9, 1)}T"],  # Convert to trillions
        'Bank Level Loss': [f"{-round(bank_losses_assets['total_loss'].median() / 1e3, 1)}M"],  # Convert to millions
        'Bank Level Loss Std': [f"{round(bank_losses_assets['total_loss'].std() / 1e6, 2)}B"],  # Std deviation for Bank Level Loss
        'Share RMBS': [round(bank_losses_assets['Share RMBs'].median() * 100, 1)],  # Median percentage
        'Share RMBS Std': [round(bank_losses_assets['Share RMBs'].std() * 100, 1)],  # Std deviation for Share RMBS
        'Share Treasury and Other': [round(bank_losses_assets['Share Treasury and Other'].median() * 100, 1)],  # Median percentage
        'Share Treasury and Other Std': [round(bank_losses_assets['Share Treasury and Other'].std() * 100, 1)],  # Std deviation
        'Share Residential Mortgage': [round(bank_losses_assets['Share Residential Mortgage'].median() * 100, 1)],  # Median percentage
        'Share Residential Mortgage Std': [round(bank_losses_assets['Share Residential Mortgage'].std() * 100, 1)],  # Std deviation
        'Share Other Loan': [round(bank_losses_assets['Share Other Loan'].median() * 100, 1)],  # Median percentage
        'Share Other Loan Std': [round(bank_losses_assets['Share Other Loan'].std() * 100, 1)],  # Std deviation
        'Loss/Asset': [round(bank_losses_assets['loss/gross_asset'].median() * 100, 1)],  # Median percentage
        'Loss/Asset Std': [round(bank_losses_assets['loss/gross_asset'].std() * 100, 1)],  # Std deviation
        'Uninsured Deposit/MM Asset': [round(uninsured_deposit_mm_asset['Uninsured_Deposit_MM_Asset'].median() * 100, 1)],  # Median percentage
        'Uninsured Deposit/MM Asset Std': [round(uninsured_deposit_mm_asset['Uninsured_Deposit_MM_Asset'].std() * 100, 1)],  # Std deviation
        'Insured Deposit Coverage Ratio': [round(insured_deposit_coverage['insured_deposit_coverage_ratio'].median() * 100, 1)],  # Median percentage
        'Insured Deposit Coverage Ratio Std': [round(insured_deposit_coverage['insured_deposit_coverage_ratio'].std() * 100, 1)],  # Std deviation
        'Number of Banks': [len(bank_losses_assets.index.unique())]  # Count of unique banks
    })

    # Rename index to 'All Banks'
    final_stats.index = [index_name]

    final_stats = final_stats.T
    
    return final_stats

In [321]:
def GSIB_bank_id():
    #GSIB = [35301,93619,229913,398668,413208,451965,476810,480228,488318,
     #497404,541101,651448,688079,722777,812164,852218,934329,1225761,
     #1443266,1456501,2182786,2362458,2489805,2531991,3066025]
    GSIB = [852218, 480228, 476810, 413208, #JP Morgan, Bank of America, Citigroup, HSBC
      2980209, 2182786, 541101, 655839, 1015560, 229913,#Barclays, Goldman Sachs, BNY Mellon, CCB COMMUNITY BANK, ICBC, Mizuho
       1456501, 722777, 35301, 925411, 497404, 3212149, #Morgan Stanley, Santander, State Street, Sumitomo Mitsui, TD Bank, UBS
      451965] #wells fargo
    return GSIB

def large_ex_GSIB_bank_id(large):
    bank_id_large_ex_GSIB = []
    for bank_id in large['Bank_ID']:
       bank_id_large_ex_GSIB.append(bank_id)
    return bank_id_large_ex_GSIB

def small_bank_id(small):
    bank_id_small = []
    for bank_id in small['Bank_ID']:
       bank_id_small.append(bank_id)
    return bank_id_small

In [322]:
GSIB = GSIB_bank_id() #list of GSIB bank IDs
df_asset_GSIB = df_asset[df_asset['Bank_ID'].isin(GSIB)] #total assets all GSIB banks
#Large non-GSIB Banks
df_asset_large_ex_GSIB = df_asset[(~df_asset['Bank_ID'].isin(GSIB)) & (df_asset['gross_asset']>1384000)] #total assets all large non-GSIB banks
large_ex_GSIB = large_ex_GSIB_bank_id(df_asset_large_ex_GSIB) #list of large non-GSIB bank IDs
#Small Banks
df_asset_small = df_asset[(~df_asset['Bank_ID'].isin(GSIB)) & (df_asset['gross_asset']<=1384000)] #total asset all small banks 
small = small_bank_id(df_asset_small)

In [323]:
df_RMBS_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(GSIB)] #RMBS for GSIB banks
df_RMBS_large_ex_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(large_ex_GSIB)] #RMBS for large non-GSIB banks
df_RMBS_small = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(small)]

In [324]:
df_RMBS_Final = df_RMBS_Final #RMBS for all banks 
df_RMBS_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(GSIB)] #RMBS for GSIB banks
df_RMBS_large_ex_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(large_ex_GSIB)] #RMBS for large non-GSIB banks
df_RMBS_small = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(small)] #RMBS for small banks

#Loans First Lien Domestic

df_loans_first_lien_domestic = df_loans_first_lien_domestic # loans first lien domestic for all banks
df_loans_first_lien_domestic_GSIB = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(GSIB)] # loans first lien domestic for all GSIB banks
df_loans_first_lien_domestic_large_ex_GSIB = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(large_ex_GSIB)] # loans first lien domestic for all large non-GSIB banks
df_loans_first_lien_domestic_small = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(small)]

df_treasury_and_others = df_treasury_and_others #treasury and others all banks 
df_treasury_and_others_GSIB = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(GSIB)] #treasury and others GSIB banks
df_treasury_and_others_large_ex_GSIB = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(large_ex_GSIB)] #treasury and others large non-GSIB baanks 
df_treasury_and_others_small = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(small)] #treasury and others small banks 

#Other Loan 

df_other_loan = df_other_loan #other loans for all banks 
df_other_loan_GSIB = df_other_loan[df_other_loan['Bank_ID'].isin(GSIB)] #other loans for all GSIB banks 
df_other_loan_large_ex_GSIB = df_other_loan[df_other_loan['Bank_ID'].isin(large_ex_GSIB)] #other loans for all large non-GSIB banks
df_other_loan_small = df_other_loan[df_other_loan['Bank_ID'].isin(small)] 

 #uninsured deposits
uninsured_deposit = uninsured_deposit #uninsured deposits for all banks
uninsured_deposit_GSIB = uninsured_deposit[uninsured_deposit['bank_ID'].isin(GSIB)] #uninsured deposits for GSIB banks
uninsured_deposit_large_ex_GSIB = uninsured_deposit[uninsured_deposit['bank_ID'].isin(large_ex_GSIB)] #uninsured deposits for large non-GSIB banks
uninsured_deposit_small = uninsured_deposit[uninsured_deposit['bank_ID'].isin(small)] #uninsured deposits for small banks

#insured deposits
insured_deposits = insured_deposit.copy() #insured deposits for all banks
insured_deposits_GSIB = insured_deposits[insured_deposits['bank_ID'].isin(GSIB)] #insured deposits for GSIB banks
insured_deposits_large_ex_GSIB = insured_deposits[insured_deposits['bank_ID'].isin(large_ex_GSIB)] #insured deposits for large non-GSIB banks
insured_deposits_small = insured_deposits[insured_deposits['bank_ID'].isin(small)]

In [325]:
bank_losses = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_Final, df_loans_first_lien_domestic,\
                                            df_treasury_and_others, df_other_loan, treasury_prices, RMBS_Multiplier, df_asset)

In [326]:
bank_losses_assets = bank_losses.copy()
bank_losses_assets_small = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_small, df_loans_first_lien_domestic_small, df_treasury_and_others_small, df_other_loan_small, treasury_prices, RMBS_Multiplier, df_asset_small)


In [327]:
bank_losses_assets_large_ex_GSIB = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_large_ex_GSIB, df_loans_first_lien_domestic_large_ex_GSIB, df_treasury_and_others_large_ex_GSIB, df_other_loan_large_ex_GSIB, treasury_prices, RMBS_Multiplier, df_asset_large_ex_GSIB)

In [328]:
bank_losses_assets_GSIB = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_GSIB, df_loans_first_lien_domestic_GSIB, df_treasury_and_others_GSIB, df_other_loan_GSIB, treasury_prices, RMBS_Multiplier, df_asset_GSIB)

In [329]:
uninsured_deposit_mm_asset_small = calculate_uninsured_deposit_mm_asset(uninsured_deposit_small, bank_losses_assets_small)
insured_deposit_coverage_small = insured_deposit_coverage_ratio(insured_deposits_small, uninsured_deposit_small, bank_losses_assets_small)
uninsured_deposit_mm_asset_large_ex_GSIB = calculate_uninsured_deposit_mm_asset(uninsured_deposit_large_ex_GSIB, bank_losses_assets_large_ex_GSIB)
insured_deposit_coverage_large_ex_GSIB = insured_deposit_coverage_large_ex_GSIB = insured_deposit_coverage_ratio(insured_deposits_large_ex_GSIB, uninsured_deposit_large_ex_GSIB, bank_losses_assets_large_ex_GSIB)
uninsured_deposit_mm_asset_GSIB = calculate_uninsured_deposit_mm_asset(uninsured_deposit_GSIB, bank_losses_assets_GSIB)
insured_deposit_coverage_GSIB = insured_deposit_coverage_ratio(insured_deposits_GSIB, uninsured_deposit_GSIB, bank_losses_assets_GSIB)
    

In [330]:
final_stats = final_statistic_table(bank_losses_assets, un_mm_ratio, coverage_ratios_df)
final_stats_small = final_statistic_table(bank_losses_assets_small, uninsured_deposit_mm_asset_small, insured_deposit_coverage_small, index_name = 'Small Banks')
final_stats_large_ex_GSIB = final_statistic_table(bank_losses_assets_large_ex_GSIB, uninsured_deposit_mm_asset_large_ex_GSIB, insured_deposit_coverage_large_ex_GSIB, index_name = 'Large Ex GSIB Banks')
final_stats_GSIB = final_statistic_table(bank_losses_assets_GSIB, uninsured_deposit_mm_asset_GSIB, insured_deposit_coverage_GSIB, index_name = 'GSIB Banks')

In [331]:
table_1 = pd.concat([final_stats, final_stats_small, final_stats_large_ex_GSIB, final_stats_GSIB], axis=1)
table_1

,All Banks,Small Banks,Large Ex GSIB Banks,GSIB Banks
Aggregate Loss,2.0T,0.1T,1.0T,0.9T
Bank Level Loss,29.7M,22.8M,318.6M,9003.4M
Bank Level Loss Std,6.52B,0.04B,4.93B,93.79B
Share RMBS,727.5,640.1,1241.6,2088.9
Share RMBS Std,1647.3,1588.2,1842.7,3070.9
Share Treasury and Other,1791.8,1920.5,1220.4,1583.0
Share Treasury and Other Std,2273.0,2343.6,1711.6,1594.4
Share Residential Mortgage,1802.9,1770.3,1962.9,2353.8
Share Residential Mortgage Std,2139.8,2163.4,2001.8,2283.3
Share Other Loan,3727.1,3649.9,4091.6,896.1
